<a href="https://colab.research.google.com/github/TokyoYoshida/GoogleColab/blob/master/playground_deep_learning_keiba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# 必要ならば以下のようにディレクトリ移動する
 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/'My Drive'/'★個人専用ファイル'/'開発用・個人サービス開発'/'GoogleColab'/

/content/drive/My Drive/★個人専用ファイル/開発用・個人サービス開発/GoogleColab


In [3]:
%ls


data/
duel_dqn_Pendulum-v0_weights.h5f
h5f/
model.txt
playground_deep_learning_keiba
playground_dqn_keiba
playground_dqn_keiba_try_layer_add_depth
playground_dqn_keiba_try_long_term_reward
playground_dqn_keiba_try_simple
playground_dqn_keiba_try_simple2
playground_dqn_keiba_try_simple_x_1
playground_dqn_study.ipynb
playground_dqn_study_my_try.ipynb
playground_dqn_study_my_try_modify_reward
playground.ipynb


In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
from pprint import pprint

In [7]:
!pip install tensorflow==2.2.0

In [0]:
!pip install -q keras

In [9]:
import keras
from keras.utils import np_utils


Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop


In [0]:
# 競馬データの作成

In [0]:
ldat = pd.read_pickle("./data/keiba_dat_res_20190611_sorted.pkl")

In [13]:
ldat.head(1)

,rank,frame_num,horse_num,name,sex,age,amount,jockey,time,diff_arrival,odds,popularity,weight,diff_weight,trainer,race_num,grade,ground,direction,unknown,distance,weather,condition,start_time,year,month,day,prev_rank,prev_frame_num,prev_horse_num,prev_name,prev_sex,prev_age,prev_amount,prev_jockey,prev_time,prev_diff_arrival,prev_odds,prev_popularity,prev_weight,prev_diff_weight,prev_trainer,prev_race_num,prev_grade,prev_ground,prev_direction,prev_unknown,prev_distance,prev_weather,prev_condition,prev_start_time,prev_year,prev_month,prev_day,winner,return,prev_time_float,start_time_float,prev_start_time_float
0,1.0,6,6,0,0,2,54.0,0,1:31.6,0,3.2,1.0,474.0,0.0,0,1,9,0,0,-,1500.0,0,0,10:40,2006,8,12,3.0,6.0,6.0,0.0,0.0,2.0,51.0,75.0,1:12.2,1.0,9.6,3.0,474.0,-2.0,0.0,1.0,9.0,0.0,0.0,-,1200.0,1.0,0.0,10:40,2006.0,7.0,29.0,True,3.2,1561165932,1561200000,1561200000


In [0]:
sdat = ldat.sample(frac=1).reset_index(drop=True)

In [0]:
x = sdat[['frame_num', 'horse_num', 'name', 'sex', 'age', 'amount',
       'jockey', 'odds', 'popularity', 'weight',
       'diff_weight', 'trainer', 'race_num', 'grade', 'ground', 'direction',
        'distance', 'weather', 'condition', 'start_time_float', 'year',
       'month', 'day', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
       'prev_name', 'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
       'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
       'prev_weight', 'prev_diff_weight', 'prev_trainer', 'prev_race_num',
       'prev_grade', 'prev_ground', 'prev_direction',
       'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float',
       'prev_year', 'prev_month', 'prev_day']].dropna(0)

In [0]:
# x = sdat[['frame_num', 'horse_num', 'sex', 'age', 'amount',
#        'odds', 'popularity', 'weight',
#        'diff_weight', 'trainer', 'grade', 'ground', 'direction',
#         'distance', 'weather', 'condition', 'start_time_float', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
#        'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
#        'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
#        'prev_weight', 'prev_diff_weight', 'prev_trainer',
#        'prev_grade', 'prev_ground', 'prev_direction',
#        'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float'
#        ]].dropna(0)

In [0]:
# x = sdat[["odds",'prev_rank','prev_diff_arrival']].dropna(0)

In [0]:
# x = sdat[["odds"]].dropna(0)

In [0]:
y = sdat[["return"]].dropna(0)

In [0]:
y_np = np_utils.to_categorical(sdat[["winner"]].dropna(0))
y = pd.DataFrame(y_np,columns=["lose","win"])

In [0]:
# 競馬データの作成 終わり

In [0]:
# 学習、テストデータの作成

In [0]:
trainlen = int(len(x)/2)

In [0]:
trainx = x.head(trainlen)
trainy = y.head(trainlen)

In [0]:
testx = x.tail(len(x)-trainlen)
testy = y.tail(len(x)-trainlen)

In [0]:
# 学習、テストデータの作成　終わり

In [0]:
#!pip uninstall -y keras

In [0]:
#!pip uninstall -y tensorflow

In [0]:
batch_size = 128
num_classes = 2
epochs = 3


In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2)) # 出力層2ノード,全結合
model.add(Activation("sigmoid"))

In [31]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               25600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1026      
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 289,282
Trainable params: 289,282
Non-trainable params: 0
________________________________________________

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])


In [33]:
history = model.fit(trainx, trainy,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(testx, testy))


Train on 229275 samples, validate on 229275 samples
Epoch 1/3
229275/229275 [==============================] - 26s 115us/step - loss: nan - accuracy: 0.9307 - val_loss: nan - val_accuracy: 0.9304
Epoch 2/3
229275/229275 [==============================] - 26s 113us/step - loss: nan - accuracy: 0.9309 - val_loss: nan - val_accuracy: 0.9304
Epoch 3/3
229275/229275 [==============================] - 26s 113us/step - loss: nan - accuracy: 0.9309 - val_loss: nan - val_accuracy: 0.9304


In [0]:
predictions = model.predict(testx)

In [40]:
print(predictions)

[[nan nan]
 [nan nan]
 [nan nan]
 ...
 [nan nan]
 [nan nan]
 [nan nan]]


In [38]:
score = model.evaluate(testx, testy, verbose=1)


229275/229275 [==============================] - 10s 44us/step


In [39]:

print("Test score", score[0])
print("Test accuracy", score[1])

Test score nan
Test accuracy 0.930393636226654
